# Payload Indexes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/configs/payload_indexes.ipynb)

Set up payload indexes for fast filtering. Covers keyword, integer, float, geo, and text indexes.

In [ ]:
!pip install -q qdrant-client sentence-transformers

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams, Distance, PointStruct,
    PayloadSchemaType, Filter, FieldCondition, MatchValue, Range,
)
from sentence_transformers import SentenceTransformer

In [ ]:
client = QdrantClient(":memory:")
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
client.create_collection(
    collection_name="products",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

# Insert sample products
products = [
    {"title": "Wireless Headphones", "category": "electronics", "price": 79},
    {"title": "Running Shoes", "category": "sports", "price": 120},
    {"title": "USB-C Hub", "category": "electronics", "price": 35},
    {"title": "Yoga Mat", "category": "sports", "price": 25},
    {"title": "Mechanical Keyboard", "category": "electronics", "price": 150},
]

points = [
    PointStruct(
        id=i,
        vector=model.encode(p["title"]).tolist(),
        payload=p,
    )
    for i, p in enumerate(products)
]
client.upsert(collection_name="products", points=points)
print(f"Loaded {len(products)} products")

In [ ]:
# Create payload indexes
client.create_payload_index(
    collection_name="products",
    field_name="category",
    field_schema=PayloadSchemaType.KEYWORD,
)

client.create_payload_index(
    collection_name="products",
    field_name="price",
    field_schema=PayloadSchemaType.INTEGER,
)

print("Created keyword index on 'category' and integer index on 'price'")

In [ ]:
# Search with filters
query_vector = model.encode("computer accessories").tolist()

response = client.query_points(
    collection_name="products",
    query=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(key="category", match=MatchValue(value="electronics")),
            FieldCondition(key="price", range=Range(gte=10, lte=100)),
        ]
    ),
    limit=10,
)
results = response.points

print("Electronics under $100:")
for r in results:
    print(f"  Score: {r.score:.4f} | {r.payload['title']} (${r.payload['price']})")